# Simulating graded electrodes

In [1]:
%pip install "pybamm[plot,cite]" -q # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
model = pybamm.lithium_ion.DFN()

In [3]:
parameter_values = pybamm.ParameterValues("Chen2020")

In [4]:
L_n = parameter_values["Negative electrode thickness [m]"]


def eps_n(x):
    return 0.2 + 0.1 * (x / L_n)


parameter_values["Negative electrode porosity"] = eps_n
parameter_values["Positive electrode porosity"] = eps_n

In [5]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sol = sim.solve([0, 3600])

In [6]:
sol.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3556.5751296553094, step=35.565751296553096)…

In [7]:
sol.plot(
    output_variables=["Negative electrode porosity", "Positive electrode porosity"]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=3556.5751296553094, step=35.565751296553096)…